# Import dependencies

In [28]:
import torch
from torch.utils.data import random_split, DataLoader
import torch.nn as nn
from torchvision.datasets import FashionMNIST
import torchvision.transforms as transforms
import matplotlib
import matplotlib.pyplot as plt
import numpy 
import time

# Data preparation

In [36]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        (0.5), (0.5)
    )
])

train_data = FashionMNIST(
    root='./data',
    train=True,
    download=True,
    transform = transform
)

test_data = FashionMNIST(
    root='./data',
    train=False,
    download=True,
    transform = transform
)

# Define VGG network

In [17]:
def vgg_block(num_convs, num_channels):
    layers = []
    for i in range(num_convs):
        layers.append(
            nn.LazyConv2d(
                out_channels=num_channels,
                kernel_size=(3, 3),
                padding=1
            )
        )
        layers.append(nn.ReLU())
    layers.append(
        nn.MaxPool2d(
            kernel_size=(2, 2),
            stride=2
        )
    )
    return nn.Sequential(*layers)
        
class VGG(nn.Module):
    def __init__(self, arch, num_classes=10):
        super(VGG, self).__init__()
        conv_blks = []
        
        for (num_convs, num_channels) in arch:
            conv_blks.append(vgg_block(num_convs, num_channels))
         
        self.net = nn.Sequential(
            *conv_blks,
            nn.Flatten(),
            nn.Linear(25088, 4096),
            nn.Linear(4096, 4096),
            nn.Linear(4096, num_classes)
        )
        
    def forward(self, x):
        return self.net(x)

# Training

## Hyperparameters

In [52]:
EPOCHS = 20
LR = 0.001
BATCH_SIZE = 16
WEIGHT_DECAY = 0.01
MOMENTUM = 0.9
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

TRAIN_RATIO = 0.75
VAL_RATIO = 1 - TRAIN_RATIO
TRAIN_SIZE = int(len(train_data) * TRAIN_RATIO)
VAL_SIZE = len(train_data) - TRAIN_SIZE

arch = ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))
model = VGG(arch).to(DEVICE)

optimizer = torch.optim.SGD(
    model.parameters(),
    lr=LR,
    weight_decay=WEIGHT_DECAY,
    momentum=MOMENTUM
)
criterion = nn.CrossEntropyLoss()

/home/quang/miniconda3/envs/deeplearning/lib/python3.12/site-packages/torch/nn/modules/lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


## Split training dataset into train and validation dataset

In [38]:
train_data, val_data = random_split(
    dataset=train_data,
    lengths=[TRAIN_SIZE, VAL_SIZE],
    generator=torch.Generator().manual_seed(42)
)

## Data loaders

In [39]:
train_loader = DataLoader(
    dataset=train_data,
    shuffle=True,
    batch_size=BATCH_SIZE
)

val_loader = DataLoader(
    dataset=val_data,
    batch_size=BATCH_SIZE
)

test_loader = DataLoader(
    dataset=test_data,
    batch_size=BATCH_SIZE
)

## For plotting

In [44]:
matplotlib.use("Agg")
H = {
    'avg_train_loss': [],
    'avg_val_loss': [],
    'train_acc': [],
    'val_acc': []
}

## Train

In [45]:
start_time = time.time()
print('[INFO] Start training ...')

for epoch in range(EPOCHS):
    total_train_loss = 0
    total_val_loss = 0
    train_acc = 0
    val_acc = 0

    # set model to train mode
    model.train()

    for batch_idx, (X, y) in enumerate(train_loader):
        X, y = X.to(DEVICE), y.to(DEVICE)
        # feed forward
        pred = model(X)
        loss = criterion(pred, y)

        # backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()
        train_acc += (pred.argmax(1) == y).type(torch.float).sum().item()

    # validation
    with torch.no_grad():
        model.eval()

        for batch_idx, (X, y) in enumerate(val_loader):
            X, y = X.to(DEVICE), y.to(DEVICE)
            pred = model(X)
            loss = criterion(pred, y)

            total_val_loss += loss.item()
            val_acc += (pred.argmax(1) == y).type(torch.float).sum().item()            

    total_train_loss /= len(train_loader.sampler)
    total_val_loss /= len(val_loader.sampler)
    train_acc /= len(train_loader.dataset)
    val_acc /= len(val_loader.dataset)
    
    H['avg_train_loss'].append(total_train_loss)
    H['avg_val_loss'].append(total_val_loss)
    H['train_acc'].append(train_acc)
    H['val_acc'].append(val_acc)

    print(f'[INFO] Epoch {epoch}/{EPOCHS}')
    print(f'Training loss: {total_train_loss:5f}, training accuracy: {train_acc:.5f}')
    print(f'Validation loss: {total_val_loss:5f}, validation accuracy: {val_acc:.5f}')


end_time = time.time()
print('[INFO] Done!')
print(f'[INFO] Training took {end_time - start_time: .4f} sec')

[INFO] Start training ...
[INFO] Epoch 0/1
Training loss: 0.143948, training accuracy: 0.09996
Validation loss: 0.144028, validation accuracy: 0.09660
[INFO] Done!
[INFO] Training took  804.1537 sec


In [51]:
plt.style.use('ggplot')
plt.figure()
plt.plot(H['avg_train_loss'], label='avg_train_loss')
plt.plot(H['avg_val_loss'], label='avg_val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.savefig('./loss.png')

plt.figure()
plt.plot(H['train_acc'], label='train_acc')
plt.plot(H['val_acc'], label='val_acc')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.savefig('./acc.png')

In [58]:
test_features, test_labels = next(iter(test_loader))
print(f"Feature batch shape: {test_features.size()}")
print(f"Labels batch shape: {test_labels.size()}")
img = test_features[0].squeeze()
label = test_labels[0]
plt.imshow(img, cmap="gray")
plt.show()
print(f"Label: {label}")

Feature batch shape: torch.Size([16, 1, 224, 224])
Labels batch shape: torch.Size([16])
Label: 9


/tmp/ipykernel_50995/220956050.py:7: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [62]:
from sklearn.metrics import classification_report
import numpy as np

with torch.no_grad():
    preds = []
    model.eval()
    for batch_idx, (X, y) in enumerate(test_loader):
        X, y = X.to(DEVICE), y.to(DEVICE)
        pred = model(X)
        loss = criterion(pred, y)
        
        preds.extend(pred.argmax(1).cpu().numpy())
    print(
        classification_report(
            y_true=test_data.targets.cpu().numpy(),
            y_pred=np.array(preds),
            target_names=test_data.classes
        )
    )

              precision    recall  f1-score   support

 T-shirt/top       0.00      0.00      0.00      1000
     Trouser       0.00      0.00      0.00      1000
    Pullover       0.00      0.00      0.00      1000
       Dress       0.00      0.00      0.00      1000
        Coat       0.00      0.00      0.00      1000
      Sandal       0.00      0.00      0.00      1000
       Shirt       0.00      0.00      0.00      1000
     Sneaker       0.00      0.00      0.00      1000
         Bag       0.00      0.00      0.00      1000
  Ankle boot       0.10      1.00      0.18      1000

    accuracy                           0.10     10000
   macro avg       0.01      0.10      0.02     10000
weighted avg       0.01      0.10      0.02     10000



/home/quang/miniconda3/envs/deeplearning/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/quang/miniconda3/envs/deeplearning/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/quang/miniconda3/envs/deeplearning/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier